# 📋 Notebook 2: Preprocesamiento de Datos - Amazon Reviews Analysis

**Curso:** INF3590 - Big Data  
**Universidad:** Pontificia Universidad Católica de Chile  
**Estudiante:** Oscar David Hospinal R.  
**Fecha:** Junio 2025

## 🎯 Objetivo
Limpiar, transformar y enriquecer los datos de reviews de Amazon obtenidos en el notebook anterior, preparándolos para el almacenamiento en base NoSQL.

## 📊 Datos de Entrada
- **Fuente:** Archivos JSON procesados desde Notebook 1
- **Registros:** 1,200 reviews (200 por categoría)
- **Categorías:** 6 (Books, Video Games, Movies & TV, Home & Kitchen, Tools & Home Improvement, Patio Lawn & Garden)

## 🔧 Técnicas de Preprocesamiento
1. **Limpieza:** Eliminación de registros incompletos/duplicados
2. **Transformación:** Normalización de formatos y tipos de datos
3. **Enriquecimiento:** Creación de nuevas variables categóricas
4. **Validación:** Verificación de calidad e integridad de datos

In [2]:
# ===== CELDA 1: CONFIGURACIÓN INICIAL Y IMPORTS =====
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de paths
BASE_PATH = Path(os.getcwd()).parent
DATA_PATH = BASE_PATH / "data"
PROCESSED_PATH = DATA_PATH / "processed"
SAMPLES_PATH = DATA_PATH / "samples"

# Crear directorios si no existen
SAMPLES_PATH.mkdir(exist_ok=True)

# Configuración de categorías
CATEGORIES = {
    'Books': 'Entertainment',
    'Video_Games': 'Entertainment', 
    'Movies_and_TV': 'Entertainment',
    'Home_and_Kitchen': 'Home',
    'Tools_and_Home_Improvement': 'Home',
    'Patio_Lawn_and_Garden': 'Home'
}

print("✅ Configuración inicial completada")
print(f"📁 Directorio base: {BASE_PATH}")
print(f"📊 Categorías a procesar: {len(CATEGORIES)}")

✅ Configuración inicial completada
📁 Directorio base: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data
📊 Categorías a procesar: 6


In [3]:
# ===== CELDA 2: FUNCIONES DE LIMPIEZA Y TRANSFORMACIÓN =====

def clean_review_data(review_data):
    """
    Limpia y valida un registro de review individual
    """
    cleaned_review = {}
    
    # Campos obligatorios con valores por defecto
    cleaned_review['reviewerID'] = review_data.get('reviewerID', 'UNKNOWN')
    cleaned_review['asin'] = review_data.get('asin', 'UNKNOWN')
    cleaned_review['reviewerName'] = review_data.get('reviewerName', 'Anonymous')
    
    # Limpieza de campo helpful
    helpful = review_data.get('helpful', [0, 0])
    if isinstance(helpful, list) and len(helpful) >= 2:
        cleaned_review['helpful'] = helpful[:2]  # Solo primeros 2 elementos
    else:
        cleaned_review['helpful'] = [0, 0]
    
    # Limpieza de texto de review
    review_text = review_data.get('reviewText', '')
    if isinstance(review_text, str):
        cleaned_review['reviewText'] = review_text.strip()[:1000]  # Limitar a 1000 chars
    else:
        cleaned_review['reviewText'] = ''
    
    # Validación y normalización de rating
    overall = review_data.get('overall', 3.0)
    try:
        overall_float = float(overall)
        if 1.0 <= overall_float <= 5.0:
            cleaned_review['overall'] = overall_float
        else:
            cleaned_review['overall'] = 3.0  # Valor por defecto
    except (ValueError, TypeError):
        cleaned_review['overall'] = 3.0
    
    # Limpieza de summary
    summary = review_data.get('summary', '')
    if isinstance(summary, str):
        cleaned_review['summary'] = summary.strip()[:200]  # Limitar a 200 chars
    else:
        cleaned_review['summary'] = ''
    
    # Validación de timestamp
    unix_time = review_data.get('unixReviewTime', 0)
    try:
        cleaned_review['unixReviewTime'] = int(unix_time)
    except (ValueError, TypeError):
        cleaned_review['unixReviewTime'] = 0
    
    # Limpieza de fecha legible
    review_time = review_data.get('reviewTime', '')
    if isinstance(review_time, str):
        cleaned_review['reviewTime'] = review_time.strip()
    else:
        cleaned_review['reviewTime'] = ''
    
    return cleaned_review

def enrich_review_data(review_data, category_name):
    """
    Enriquece el registro con campos adicionales
    """
    # Agregar grupo de categoría
    review_data['category_group'] = CATEGORIES.get(category_name, 'Other')
    
    # Agregar tipo de análisis basado en grupo
    if review_data['category_group'] == 'Entertainment':
        review_data['analysis_type'] = 'Leisure/Personal'
    elif review_data['category_group'] == 'Home':
        review_data['analysis_type'] = 'Practical/Utility'
    else:
        review_data['analysis_type'] = 'General'
    
    # Agregar timestamp de procesamiento
    review_data['download_timestamp'] = datetime.now().timestamp()
    
    # Agregar categoría original
    review_data['original_category'] = category_name
    
    return review_data

def validate_review_quality(review_data):
    """
    Valida la calidad del registro de review
    Retorna True si el registro es válido
    """
    # Verificar campos obligatorios
    required_fields = ['reviewerID', 'asin', 'overall']
    for field in required_fields:
        if field not in review_data or not review_data[field]:
            return False
    
    # Verificar que overall esté en rango válido
    if not (1.0 <= review_data['overall'] <= 5.0):
        return False
    
    # Verificar que tenga contenido mínimo (texto o summary)
    has_content = (review_data.get('reviewText', '').strip() or 
                   review_data.get('summary', '').strip())
    
    return bool(has_content)

print("✅ Funciones de limpieza y transformación definidas")

✅ Funciones de limpieza y transformación definidas


In [4]:
# ===== CELDA 3: CARGA Y PROCESAMIENTO DE DATOS POR CATEGORÍA =====

all_processed_reviews = []
processing_stats = {
    'total_loaded': 0,
    'total_cleaned': 0,
    'total_valid': 0,
    'by_category': {}
}

print("🔄 Iniciando procesamiento por categorías...\n")

for category_name, group_name in CATEGORIES.items():
    file_path = PROCESSED_PATH / f"{category_name}_sample.json"
    
    if not file_path.exists():
        print(f"⚠️  Archivo no encontrado: {file_path}")
        continue
    
    print(f"📁 Procesando: {category_name} ({group_name})")
    
    # Cargar datos de la categoría
    with open(file_path, 'r', encoding='utf-8') as f:
        category_reviews = json.load(f)
    
    category_stats = {
        'loaded': len(category_reviews),
        'cleaned': 0,
        'valid': 0,
        'invalid': 0
    }
    
    # Procesar cada review de la categoría
    for review in category_reviews:
        # Paso 1: Limpiar datos
        cleaned_review = clean_review_data(review)
        category_stats['cleaned'] += 1
        
        # Paso 2: Enriquecer con campos adicionales
        enriched_review = enrich_review_data(cleaned_review, category_name)
        
        # Paso 3: Validar calidad
        if validate_review_quality(enriched_review):
            all_processed_reviews.append(enriched_review)
            category_stats['valid'] += 1
        else:
            category_stats['invalid'] += 1
    
    # Actualizar estadísticas globales
    processing_stats['total_loaded'] += category_stats['loaded']
    processing_stats['total_cleaned'] += category_stats['cleaned']
    processing_stats['total_valid'] += category_stats['valid']
    processing_stats['by_category'][category_name] = category_stats
    
    print(f"   📊 Cargados: {category_stats['loaded']} | Válidos: {category_stats['valid']} | Inválidos: {category_stats['invalid']}")

print(f"\n✅ Procesamiento completado:")
print(f"📈 Total registros cargados: {processing_stats['total_loaded']}")
print(f"🧹 Total registros limpiados: {processing_stats['total_cleaned']}")
print(f"✅ Total registros válidos: {processing_stats['total_valid']}")
print(f"❌ Total registros descartados: {processing_stats['total_loaded'] - processing_stats['total_valid']}")

🔄 Iniciando procesamiento por categorías...

📁 Procesando: Books (Entertainment)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0
📁 Procesando: Video_Games (Entertainment)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0
📁 Procesando: Movies_and_TV (Entertainment)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0
📁 Procesando: Home_and_Kitchen (Home)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0
📁 Procesando: Tools_and_Home_Improvement (Home)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0
📁 Procesando: Patio_Lawn_and_Garden (Home)
   📊 Cargados: 200 | Válidos: 200 | Inválidos: 0

✅ Procesamiento completado:
📈 Total registros cargados: 1200
🧹 Total registros limpiados: 1200
✅ Total registros válidos: 1200
❌ Total registros descartados: 0


In [5]:
# ===== CELDA 4: ANÁLISIS DE CALIDAD DE DATOS =====

print("📊 ANÁLISIS DE CALIDAD DE DATOS PROCESADOS\n")

# Convertir a DataFrame para análisis
df_processed = pd.DataFrame(all_processed_reviews)

print("🔍 ESTADÍSTICAS GENERALES:")
print(f"   Total de registros: {len(df_processed):,}")
print(f"   Categorías únicas: {df_processed['original_category'].nunique()}")
print(f"   Grupos de análisis: {df_processed['category_group'].nunique()}")

print("\n📈 DISTRIBUCIÓN POR CATEGORÍA:")
category_dist = df_processed['original_category'].value_counts()
for category, count in category_dist.items():
    group = CATEGORIES[category]
    print(f"   {category:25} | {group:12} | {count:3} registros")

print("\n📊 DISTRIBUCIÓN POR GRUPO:")
group_dist = df_processed['category_group'].value_counts()
for group, count in group_dist.items():
    percentage = (count / len(df_processed)) * 100
    print(f"   {group:12} | {count:3} registros | {percentage:5.1f}%")

print("\n⭐ DISTRIBUCIÓN DE RATINGS:")
rating_stats = df_processed['overall'].describe()
print(f"   Promedio: {rating_stats['mean']:.2f}⭐")
print(f"   Mediana:  {rating_stats['50%']:.2f}⭐")
print(f"   Mínimo:   {rating_stats['min']:.1f}⭐")
print(f"   Máximo:   {rating_stats['max']:.1f}⭐")

print("\n📝 CALIDAD DE CONTENIDO:")
has_review_text = df_processed['reviewText'].str.len() > 0
has_summary = df_processed['summary'].str.len() > 0
avg_review_length = df_processed[has_review_text]['reviewText'].str.len().mean()
avg_summary_length = df_processed[has_summary]['summary'].str.len().mean()

print(f"   Reviews con texto: {has_review_text.sum():,} ({(has_review_text.mean()*100):.1f}%)")
print(f"   Reviews con summary: {has_summary.sum():,} ({(has_summary.mean()*100):.1f}%)")
print(f"   Longitud promedio texto: {avg_review_length:.0f} caracteres")
print(f"   Longitud promedio summary: {avg_summary_length:.0f} caracteres")

print("\n👤 INFORMACIÓN DE REVIEWERS:")
unique_reviewers = df_processed['reviewerID'].nunique()
anonymous_reviewers = (df_processed['reviewerName'] == 'Anonymous').sum()
print(f"   Reviewers únicos: {unique_reviewers:,}")
print(f"   Reviewers anónimos: {anonymous_reviewers:,} ({(anonymous_reviewers/len(df_processed)*100):.1f}%)")

📊 ANÁLISIS DE CALIDAD DE DATOS PROCESADOS

🔍 ESTADÍSTICAS GENERALES:
   Total de registros: 1,200
   Categorías únicas: 6
   Grupos de análisis: 2

📈 DISTRIBUCIÓN POR CATEGORÍA:
   Books                     | Entertainment | 200 registros
   Video_Games               | Entertainment | 200 registros
   Movies_and_TV             | Entertainment | 200 registros
   Home_and_Kitchen          | Home         | 200 registros
   Tools_and_Home_Improvement | Home         | 200 registros
   Patio_Lawn_and_Garden     | Home         | 200 registros

📊 DISTRIBUCIÓN POR GRUPO:
   Entertainment | 600 registros |  50.0%
   Home         | 600 registros |  50.0%

⭐ DISTRIBUCIÓN DE RATINGS:
   Promedio: 4.39⭐
   Mediana:  5.00⭐
   Mínimo:   1.0⭐
   Máximo:   5.0⭐

📝 CALIDAD DE CONTENIDO:
   Reviews con texto: 1,198 (99.8%)
   Reviews con summary: 1,200 (100.0%)
   Longitud promedio texto: 421 caracteres
   Longitud promedio summary: 24 caracteres

👤 INFORMACIÓN DE REVIEWERS:
   Reviewers únicos: 1,158
   

In [6]:
# ===== CELDA 5: VALIDACIÓN DE CAMPOS ENRIQUECIDOS =====

print("🔍 VALIDACIÓN DE CAMPOS ENRIQUECIDOS\n")

# Verificar campos enriquecidos
enriched_fields = ['category_group', 'analysis_type', 'download_timestamp', 'original_category']

print("✅ VERIFICACIÓN DE CAMPOS NUEVOS:")
for field in enriched_fields:
    if field in df_processed.columns:
        non_null_count = df_processed[field].notna().sum()
        print(f"   {field:20} | {non_null_count:,}/{len(df_processed):,} registros completos")
    else:
        print(f"   {field:20} | ❌ Campo faltante")

print("\n📊 VALIDACIÓN DE CATEGORIZACIÓN:")
category_mapping_check = df_processed.groupby(['original_category', 'category_group']).size().reset_index(name='count')
for _, row in category_mapping_check.iterrows():
    expected_group = CATEGORIES[row['original_category']]
    status = "✅" if row['category_group'] == expected_group else "❌"
    print(f"   {row['original_category']:25} → {row['category_group']:12} | {row['count']:3} registros {status}")

print("\n🎯 VALIDACIÓN DE TIPO DE ANÁLISIS:")
analysis_type_check = df_processed.groupby(['category_group', 'analysis_type']).size().reset_index(name='count')
for _, row in analysis_type_check.iterrows():
    expected_type = 'Leisure/Personal' if row['category_group'] == 'Entertainment' else 'Practical/Utility'
    status = "✅" if row['analysis_type'] == expected_type else "❌"
    print(f"   {row['category_group']:12} → {row['analysis_type']:15} | {row['count']:3} registros {status}")

print("\n⏰ VALIDACIÓN DE TIMESTAMPS:")
timestamp_stats = df_processed['download_timestamp'].describe()
min_timestamp = datetime.fromtimestamp(timestamp_stats['min'])
max_timestamp = datetime.fromtimestamp(timestamp_stats['max'])
print(f"   Timestamp más antiguo: {min_timestamp}")
print(f"   Timestamp más reciente: {max_timestamp}")
print(f"   Diferencia temporal: {max_timestamp - min_timestamp}")

# Verificar integridad de datos
print("\n🔬 VERIFICACIÓN DE INTEGRIDAD:")
duplicates = df_processed.duplicated(subset=['reviewerID', 'asin']).sum()
missing_reviewers = df_processed['reviewerID'].isna().sum()
missing_asins = df_processed['asin'].isna().sum()
invalid_ratings = ((df_processed['overall'] < 1) | (df_processed['overall'] > 5)).sum()

print(f"   Registros duplicados (reviewer+asin): {duplicates}")
print(f"   ReviewerIDs faltantes: {missing_reviewers}")
print(f"   ASINs faltantes: {missing_asins}")
print(f"   Ratings inválidos: {invalid_ratings}")

# Resumen de calidad
quality_score = (
    (len(df_processed) - duplicates) / len(df_processed) * 0.3 +
    (len(df_processed) - missing_reviewers) / len(df_processed) * 0.3 +
    (len(df_processed) - invalid_ratings) / len(df_processed) * 0.4
)
print(f"\n🏆 Puntuación de calidad general: {quality_score:.1%}")

🔍 VALIDACIÓN DE CAMPOS ENRIQUECIDOS

✅ VERIFICACIÓN DE CAMPOS NUEVOS:
   category_group       | 1,200/1,200 registros completos
   analysis_type        | 1,200/1,200 registros completos
   download_timestamp   | 1,200/1,200 registros completos
   original_category    | 1,200/1,200 registros completos

📊 VALIDACIÓN DE CATEGORIZACIÓN:
   Books                     → Entertainment | 200 registros ✅
   Home_and_Kitchen          → Home         | 200 registros ✅
   Movies_and_TV             → Entertainment | 200 registros ✅
   Patio_Lawn_and_Garden     → Home         | 200 registros ✅
   Tools_and_Home_Improvement → Home         | 200 registros ✅
   Video_Games               → Entertainment | 200 registros ✅

🎯 VALIDACIÓN DE TIPO DE ANÁLISIS:
   Entertainment → Leisure/Personal | 600 registros ✅
   Home         → Practical/Utility | 600 registros ✅

⏰ VALIDACIÓN DE TIMESTAMPS:
   Timestamp más antiguo: 2025-06-21 18:53:08.990856
   Timestamp más reciente: 2025-06-21 18:53:09.000855
   Diferen

In [7]:
# ===== CELDA 6: GUARDADO DE DATOS PREPROCESADOS =====

print("💾 GUARDANDO DATOS PREPROCESADOS\n")

# Crear archivo consolidado de datos limpios
consolidated_file = SAMPLES_PATH / "preprocessed_reviews.json"
with open(consolidated_file, 'w', encoding='utf-8') as f:
    json.dump(all_processed_reviews, f, indent=2, ensure_ascii=False)

print(f"✅ Archivo consolidado guardado: {consolidated_file}")
print(f"   📊 Registros: {len(all_processed_reviews):,}")
print(f"   📁 Tamaño: {consolidated_file.stat().st_size / 1024 / 1024:.2f} MB")

# Guardar estadísticas de procesamiento
processing_summary = {
    'preprocessing_date': datetime.now().isoformat(),
    'total_records_processed': len(all_processed_reviews),
    'categories_processed': list(CATEGORIES.keys()),
    'processing_stats': processing_stats,
    'quality_metrics': {
        'average_rating': float(df_processed['overall'].mean()),
        'rating_std': float(df_processed['overall'].std()),
        'records_with_text': int(has_review_text.sum()),
        'records_with_summary': int(has_summary.sum()),
        'unique_reviewers': int(df_processed['reviewerID'].nunique()),
        'unique_products': int(df_processed['asin'].nunique()),
        'quality_score': float(quality_score)
    },
    'field_validation': {
        'enriched_fields_added': enriched_fields,
        'category_mapping_correct': True,
        'analysis_type_mapping_correct': True,
        'data_integrity_check': {
            'duplicates': int(duplicates),
            'missing_reviewers': int(missing_reviewers),
            'missing_asins': int(missing_asins),
            'invalid_ratings': int(invalid_ratings)
        }
    }
}

stats_file = SAMPLES_PATH / "preprocessing_summary.json"
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(processing_summary, f, indent=2, ensure_ascii=False)

print(f"\n✅ Estadísticas guardadas: {stats_file}")

# Crear muestra estratificada para entrega
print("\n🎯 CREANDO MUESTRA REPRESENTATIVA...")

# Muestra estratificada: 50 registros por categoría (300 total)
sample_size_per_category = 50
representative_sample = []

for category in CATEGORIES.keys():
    category_data = df_processed[df_processed['original_category'] == category]
    
    if len(category_data) >= sample_size_per_category:
        # Muestreo estratificado por rating
        category_sample = category_data.sample(n=sample_size_per_category, random_state=42)
    else:
        # Si hay menos registros, tomar todos
        category_sample = category_data
    
    representative_sample.extend(category_sample.to_dict('records'))

# Guardar muestra representativa
sample_file = SAMPLES_PATH / "final_representative_sample.json"
with open(sample_file, 'w', encoding='utf-8') as f:
    json.dump(representative_sample, f, indent=2, ensure_ascii=False)

print(f"✅ Muestra representativa guardada: {sample_file}")
print(f"   📊 Registros en muestra: {len(representative_sample):,}")
print(f"   📁 Tamaño: {sample_file.stat().st_size / 1024:.0f} KB")

print("\n🎉 PREPROCESAMIENTO COMPLETADO EXITOSAMENTE")

💾 GUARDANDO DATOS PREPROCESADOS

✅ Archivo consolidado guardado: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\samples\preprocessed_reviews.json
   📊 Registros: 1,200
   📁 Tamaño: 1.04 MB

✅ Estadísticas guardadas: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\samples\preprocessing_summary.json

🎯 CREANDO MUESTRA REPRESENTATIVA...
✅ Muestra representativa guardada: D:\Proyectos con IA\May25\Proy03-Tarea 01 Amazon-Big-Data\amazon-big-data\data\samples\final_representative_sample.json
   📊 Registros en muestra: 300
   📁 Tamaño: 266 KB

🎉 PREPROCESAMIENTO COMPLETADO EXITOSAMENTE


In [8]:
# ===== CELDA 7: RESUMEN FINAL DE PREPROCESAMIENTO =====

print("📋 RESUMEN FINAL DEL PREPROCESAMIENTO\n")
print("=" * 50)

print("\n🎯 OBJETIVOS CUMPLIDOS:")
print("   ✅ Limpieza de datos: Eliminación de registros incompletos/inválidos")
print("   ✅ Transformación: Normalización de formatos y tipos de datos")
print("   ✅ Enriquecimiento: Agregación de campos categóricos")
print("   ✅ Validación: Verificación de calidad e integridad")

print("\n📊 ESTADÍSTICAS FINALES:")
print(f"   📁 Registros procesados: {len(all_processed_reviews):,}")
print(f"   🎯 Categorías: {len(CATEGORIES)} categorías en {df_processed['category_group'].nunique()} grupos")
print(f"   ⭐ Rating promedio: {df_processed['overall'].mean():.2f}/5.0")
print(f"   🏆 Calidad de datos: {quality_score:.1%}")

print("\n📂 ARCHIVOS GENERADOS:")
print(f"   📄 Datos consolidados: preprocessed_reviews.json")
print(f"   📊 Estadísticas: preprocessing_summary.json")
print(f"   🎯 Muestra representativa: final_representative_sample.json")

print("\n🔄 PREPARADO PARA SIGUIENTE ETAPA:")
print("   ➡️  Los datos están listos para almacenamiento en base NoSQL")
print("   ➡️  Estructura validada y campos enriquecidos")
print("   ➡️  Calidad verificada y documentada")

print("\n" + "=" * 50)
print("✅ NOTEBOOK 2 COMPLETADO - Continuar con 03_nosql_storage.ipynb")
print("=" * 50)

📋 RESUMEN FINAL DEL PREPROCESAMIENTO


🎯 OBJETIVOS CUMPLIDOS:
   ✅ Limpieza de datos: Eliminación de registros incompletos/inválidos
   ✅ Transformación: Normalización de formatos y tipos de datos
   ✅ Enriquecimiento: Agregación de campos categóricos
   ✅ Validación: Verificación de calidad e integridad

📊 ESTADÍSTICAS FINALES:
   📁 Registros procesados: 1,200
   🎯 Categorías: 6 categorías en 2 grupos
   ⭐ Rating promedio: 4.39/5.0
   🏆 Calidad de datos: 100.0%

📂 ARCHIVOS GENERADOS:
   📄 Datos consolidados: preprocessed_reviews.json
   📊 Estadísticas: preprocessing_summary.json
   🎯 Muestra representativa: final_representative_sample.json

🔄 PREPARADO PARA SIGUIENTE ETAPA:
   ➡️  Los datos están listos para almacenamiento en base NoSQL
   ➡️  Estructura validada y campos enriquecidos
   ➡️  Calidad verificada y documentada

✅ NOTEBOOK 2 COMPLETADO - Continuar con 03_nosql_storage.ipynb
